# Changes in residential UPRNs in the North East Region

This workbook includes two tools - one creates static matplotlib maps of the NE region, using text inputs for years and the area you want to focus on. The second tool does the same thing using an interactive map and panel widgets.

Author - Samantha Iacob

Last Updated - 10/5/2024

The cells from here on will require geopackages to be installed on your machine! A guide to installing the modules you'll need is located [here on the ONS geospatial training repository on GitHub](https://github.com/ONSgeo/geospatial-training/blob/master/_docs/guides/python_install_anaconda.md).

Additionally you will need Panel and Folium (install using *pip install* if needed)

In [ ]:
#Run this block first
import panel as pn
pn.extension(design = 'material')
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn' (turns off an annoying warning)
import geopandas as gpd
import shapely
import matplotlib.pyplot as plt
import mapclassify
from mapclassify import Quantiles, UserDefined
from matplotlib import cm
import numpy as np
from pathlib import Path
%matplotlib inline
BASE = Path('.').resolve() #Set path
import folium # additional modules
from pyproj import CRS
import requests
from folium.features import GeoJsonPopup, GeoJsonTooltip

Loading the data needed - a shapefile which contains the stats and geodata for mapping.

In [ ]:
msoa_stats = gpd.read_file("data/ne_msoa_uprns.shp").dropna() # importing data shapefile
msoa_23_bg = msoa_stats.dropna().assign(group = 1) # creating the background image for the first tool.
msoa_23_bg = msoa_23_bg.dissolve(by = 'group')

Press run in the cell below for the first chart. It will ask for starting year, end year and area as inputs

In [ ]:
def change_pct(start_year, end_year, area, gdf = msoa_stats):
    if area != 'North East Region':
        gdf = gdf[gdf.MSOA21NM.str.startswith(area)]
    #gdf['change'] = round((gdf[f'UPRNs_{end_year}'] / gdf[f'UPRNs_{start_year}'] - 1) * 100, 2) # incorrect % formula 
    gdf['change'] = ((gdf[f'UPRNs_{end_year}']-gdf[f'UPRNs_{start_year}']) / gdf[f'UPRNs_{start_year}']*100)
    return gdf
#creating a list of areas with the MSOA number stripped out
area_list = sorted(msoa_stats['MSOA21NM'].str[:-4].unique().tolist())
area_list.insert(0, 'North East Region')

#takes two years and a region as input
year_list = [2020,2021,2022,2023,2024]
while True:
    start_year = int(input(f'Enter starting year between {year_list[0]} and {year_list[-2]}: '))
    end_year = int(input(f'Enter ending year between {start_year + 1} and {year_list[-1]}: '))
    if start_year in year_list and end_year in year_list and end_year > start_year:
        break
while True:
    area = input('Area name (Leave blank for all or type area name, \'L\' for list): ')
    if area.upper() == 'L':
        count = 0
        for area in area_list:
            print(f"{count} : {area}")
            count += 1
    elif area == "" or area == "0":
        area = 'North East Region'
        break
    elif area in area_list:
            break
    
    try: 
        area = area_list[int(area)]
    except ValueError:
        pass
    else:
        break
        
if area == 'North East Region':
    line = 0.2
else:
    line = 0.75
        
fig, ax = plt.subplots(figsize=(15, 15))
if area == "North East Region":
    msoa_23_bg.plot(ax = ax, color = 'grey', edgecolor = 'black', linewidth = 0.5)
    fig.text(0.22,0.131,'Contains OS data © Crown copyright 2024')
    fig.text(0.22,0.116, 'Source: ONS, licensed under the open Government license v3.0')
    
try: 
    ax = change_pct(start_year,end_year, area, msoa_stats).plot(column='change',
                                                      linewidth = line,
                                                      edgecolor = 'k',
                                                      cmap = 'viridis',
                                                      scheme="User_Defined", 
                                                      classification_kwds={'bins':[-30,-10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 30]}, 
                                                      #scheme='Equal_Interval',
                                                      #k = 5,
                                                      legend=True,
                                                      legend_kwds = {'title' : 'Change (%)'},
                                                      missing_kwds = {'color' : 'grey', 'label' : 'missing values'},
                                                      ax=ax)
except: 
    print(f'/nError - no information available for {area} in time period')

plt.title(f'Percentage change in {area} residential UPRNs, {start_year} - {end_year}', size = 12)

The cell below is the code for the second tool. Please run it and the cell below (output will open in a new tab)

In [ ]:
gdf = msoa_stats.copy()
gdf['geometry'] = gdf['geometry'].simplify(25) #speeding up performance at the cost of some appearance.

#creating panel features
# As I want to use two dates which have been saved as integers, the IntRange Slider is a good choice
year_slider = pn.widgets.IntRangeSlider(name='Year Range', 
                                        start=2020, 
                                        end=2024, 
                                        value=(2020, 2024), 
                                        value_throttled=(2020,2024)
                                       )

#creating a list and a drop down menu based off the list.
msoa_list = sorted(gdf['MSOA21NM'].str[:-4].unique().tolist())   
msoa_list.insert(0, 'All North East')
msoa_selector = pn.widgets.Select(name='Area', options=msoa_list)

# Function to use the details from these widgets to generate the change column
df = gdf
def year_change(gdf = gdf, year = year_slider, area = msoa_selector):
    global df
    if area != 'All North East':
        gdf = gdf[gdf.MSOA21NM.str.startswith(area)]
        df = gdf[gdf.MSOA21NM.str.startswith(area)]
    gdf['change'] = round(((gdf[f'UPRNs_{year[1]}']-gdf[f'UPRNs_{year[0]}']) / gdf[f'UPRNs_{year[0]}']) * 100,2)
    df = df[['MSOA21NM', 'MSOA21CD','UPRNs_2020','UPRNs_2021','UPRNs_2022','UPRNs_2023', 'UPRNs_2024']]
    return years_map(gdf)

#Defining a function which creates the map and passes it back to panel as a pane.

def years_map(map_gdf):
    
    m = map_gdf.dropna().explore("change",
                        cmap = "viridis",
                        tiles = "CartoDB positron",
                        tooltip = ["MSOA21NM", f"UPRNs_{year_slider.value[0]}", f"UPRNs_{year_slider.value[1]}", "change"],
                        scheme = 'equalinterval',
                        k = 8,
                        style_kwds = {
                                      'stroke' : True, 
                                      'color' : "Black", 
                                      'weight' : 1.0, 
                                      'opacity' : 1,
                                      'fillOpacity' : 0.7},
                        tooltip_kwds = {'aliases' : ["MSOA:", 
                                                    f"{year_slider.value[0]} residential UPRNs:", 
                                                    f"{year_slider.value[1]} residential UPRNs:", 
                                                    "% change:"]
                                        },
                        legend_kwds = {'caption' : f'Change (%) from {year_slider.value[0]} to {year_slider.value[1]}', 
                                      },
                        map_kwds = {'dragging' : True, 
                                    'scrollWheelZoom' : True}, #this is a detailed map so it's nice to give ppl more control
                        zoom_control=True 
                        )
    return pn.Column(
        pn.pane.HTML(m._repr_html_(), sizing_mode = "scale_both"), # necessary to make this display correctly in pane
    )

#calculating country and region yearly figures for some tables to display with the map
quick_df = pd.read_csv("data/e_regions.csv")

all_eng = quick_df.groupby(['Year']).agg({'UPRN_count':'sum'})
all_ne = quick_df[quick_df.name == 'North East'].groupby(['Year']).agg({'UPRN_count' : 'sum'})

def pct_change(end, start):
    'takes two integers and determines the % change between them'
    return (end - start) / start * 100

def pct_maker(df):
    '''
    Creates a table which shows the % change in UPRNs by year.
    Takes a dataframe as an argument.
    Needs 2 columns, Year and UPRN_count. Year is an index and an int. UPRN count
    is an int.
    '''
    df=df.copy()
    new_df = pd.DataFrame()
    counter = 0
    for year in range(df.index.min(), df.index.max()+1):
        new_df[year] = counter
        counter += 1
    for year in range(df.index.min(), df.index.max()+1):
        counter = df.index.min()
        while counter < df.index.max()+1:
            end = df.loc[counter, 'UPRN_count']
            start = df.loc[year, 'UPRN_count']
            new_df.at[year, counter] = round(pct_change(end,start),2)
            counter += 1
    new_df = new_df.rename_axis('base year')
    return new_df  

#Creating two dataframes containing yearly % changes against all baseline years
ne_change = pct_maker(all_ne)
eng_change = pct_maker(all_eng)

#Setting up tabulator panes for the dash and formatting them so they look nice
from bokeh.models.widgets.tables import StringFormatter
ne_data = pn.widgets.Tabulator(ne_change, formatters = {'base year' : StringFormatter()}, theme = 'modern',  layout='fit_data_table')
eng_data = pn.widgets.Tabulator(eng_change, formatters = {'base year' : StringFormatter()}, theme = 'modern',  layout='fit_data_table')

#laying out dash
layout = pn.interact(year_change)
layout_data = pn.Column(f'## % change in residential UPRNs, all North East, from base year',
                        ne_data,
                        f'## % change in residential UPRNs, all England, from base year',
                        eng_data)
dash = pn.Column(pn.Row(f'# Percentage change in residential UPRNs in North East region between selected years', layout[0]), 
                 pn.Row(layout[1], layout_data))

Run the below to open the interactive map application in a new browser tab.

In [ ]:
dash.show()